# Hypergraph Neural Networks

This tutorial illustrates what is hypergraph and how to build a Hypergraph Neural Network using DGL's sparse matrix APIs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dglai/Thewebconf2023-Tutorial/blob/main/Hypergraph%20neural%20networks.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/dmlc/dgl/blob/master/notebooks/sparse/hgnn.ipynb)

In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Uncomment below to install required packages. If the CUDA version is not 11.6,
# check the https://www.dgl.ai/pages/start.html to find the supported CUDA
# version and corresponding command to install DGL.
!pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html > /dev/null
#!pip install torchmetrics > /dev/null

try:
    import dgl
    installed = True
except ImportError:
    installed = False
print("DGL installed!" if installed else "Failed to install DGL!")

DGL installed!


## Hypergraphs

A [hypergraph](https://en.wikipedia.org/wiki/Hypergraph) consists of *nodes* and *hyperedges*.  Contrary to edges in graphs, a *hyperedge* can connect arbitrary number of nodes.  For instance, the following figure shows a hypergraph with 11 nodes and 5 hyperedges drawn in different colors.
![](https://data.dgl.ai/tutorial/img/hgnn/hypergraph4.PNG)

Hypergraphs are particularly useful when the relationships between data points within the dataset is not binary.  For instance, more than two products can be co-purchased together in an e-commerce system, so the relationship of co-purchase is $n$-ary rather than binary, and therefore it is better described as a hypergraph rather than a normal graph.  Other examples include:
* Co-author relationships
* Events involving multiple entities

A hypergraph is usually characterized by its *incidence matrix* $H$, whose rows represent nodes and columns represent hyperedges.  An entry $H_{ij}$ is 1 if hyperedge $j$ includes node $i$, or 0 otherwise.  For example, the hypergraph in the figure above can be characterized by a $11 \times 5$ matrix as follows:

$$
H = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 \\
1 & 1 & 0 & 1 & 1 \\
0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
1 & 0 & 1 & 1 & 1 \\
0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 1 & 0 \\
0 & 1 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
$$

One can construct the hypergraph incidence matrix by specifying two tensors `nodes` and `hyperedges`, where the node ID `nodes[i]` belongs to the hyperedge ID `hyperedges[i]` for all `i`.  In the case above, the incidence matrix can be constructed below.

In [3]:
import dgl.sparse as dglsp
import torch

H = dglsp.spmatrix(
    torch.LongTensor([[0, 1, 2, 2, 2, 2, 3, 4, 5, 5, 5, 5, 6, 7, 7, 8, 8, 9, 9, 10],
                      [0, 0, 0, 1, 3, 4, 2, 1, 0, 2, 3, 4, 2, 1, 3, 1, 3, 2, 4, 4]])
)

print(H.to_dense())

tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 1., 1., 1.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 1.]])


The degree of a node in a hypergraph is defined as the number of hyperedges including the node.  Similarly, the degree of a hyperedge in a hypergraph is defined as the number of nodes included by the hyperedge.  In the example above, the hyperedge degrees can be computed by the sum of row vectors (i.e. all 4), while the node degree can be computed by the sum of column vectors.

In [ ]:
node_degrees = H.sum(1)
print("Node degrees", node_degrees)

hyperedge_degrees = H.sum(0)
print("Hyperedge degrees", hyperedge_degrees)

Node degrees tensor([1., 1., 4., 1., 1., 4., 1., 2., 2., 2., 1.])
Hyperedge degrees tensor([4., 4., 4., 4., 4.])


### Exercise

We would like to apply a hypergraph neural network on a co-citation dataset.  The co-citation dataset is based on Cora.  Since Cora is a citation network (i.e. a graph), we would like to construct a hypergraph from it, where the hyperedges represent *co-citations*: for each paper we construct a hyperedge that includes all the other papers it cited, as well as the paper itself.

![](https://data.dgl.ai/tutorial/img/hgnn/equiv.PNG)

How do we construct the incidence matrix of such a hypergraph from the adjacency matrix of the original graph?

In [ ]:
def load_data():
    dataset = dgl.data.CoraGraphDataset()
    graph = dataset[0]
    adj = graph.adj().coalesce()

    X = graph.ndata['feat']     # node input features
    Y = graph.ndata['label']    # node labels
    train_mask = graph.ndata["train_mask"]
    val_mask = graph.ndata["val_mask"]
    test_mask = graph.ndata["test_mask"]

    # INSERT YOUR CODE HERE...
    H = ...
    return H, X, Y, dataset.num_classes, train_mask, val_mask, test_mask

## Hypergraph Neural Network (HGNN) Layer

The [HGNN layer](https://arxiv.org/pdf/1809.09401.pdf) is one of the earliest hypergraph neural network formulations.  Like GNNs, an HGNN aims to learn node representations by combining the topology as well as the features of the local hypergraph neighborhood.  It is also one of the earliest hypergraph message passing networks, where each node's representation is updated with the representations of its local neighborhood.

An HGNN layer is defined as:

$$f(X^{(l)}, H; W^{(l)}) = \sigma(L X^{(l)} W^{(l)})$$$$L = D_v^{-1/2} H B D_e^{-1} H^\top D_v^{-1/2}$$

where

* $H \in \mathbb{R}^{N \times M}$ is the incidence matrix of hypergraph with $N$ nodes and $M$ hyperedges.
* $D_v \in \mathbb{R}^{N \times N}$ is a diagonal matrix representing node degrees, whose $i$-th diagonal element is $\sum_{j=1}^M H_{ij}$.
* $D_e \in \mathbb{R}^{M \times M}$ is a diagonal matrix representing hyperedge degrees, whose $j$-th diagonal element is $\sum_{i=1}^N H_{ij}$.
* $B \in \mathbb{R}^{M \times M}$ is a diagonal matrix representing the hyperedge weights, whose $j$-th diagonal element is the weight of $j$-th hyperedge.  In our example, $B$ is an identity matrix.

Extension of the HGNN framework include learning the hyperedge representations as well.  Basically, a node's representation is updated with its incident hyperedges, whereas a hyperedge's representation is updated with its member nodes.  Examples of such framework include
* [HGNN+: General Hypergraph Neural Networks (Gao et al.)](https://ieeexplore.ieee.org/document/9795251) (We will go over this if we have extra time)
* [Learning over families of sets (Srinivasan et al.)](https://arxiv.org/pdf/2101.07773.pdf)

### Exercise

Could you implement the `HypergraphConv` module that computes $L X^{(l)} W^{(l)}$ above, given what you have learned in the previous tutorials?

In [ ]:
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F

class HypergraphConv(nn.Module):
    def __init__(self, H, in_size, out_size):
        super().__init__()
        # INSERT YOUR CODE HERE...
        
    def forward(self, X):
        # REPLACE WITH YOUR CODE HERE...
        return X

The following builds a two-layer HGNN from the `HypergraphConv` module above.

In [ ]:
class HGNN(nn.Module):
    def __init__(self, H, in_size, out_size, hidden_dims=16):
        super().__init__()

        self.conv1 = HypergraphConv(H, in_size, hidden_dims)
        self.conv2 = HypergraphConv(H, hidden_dims, out_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, X):
        X = self.conv1(self.dropout(X))
        X = F.relu(X)
        X = self.conv2(self.dropout(X))
        return X

## Training and evaluation

This tutorial focuses on the node classification task: given the co-citation dataset, as well as the labels of a subset of nodes, predict the labels of the rest of the nodes.  This is akin to semi-supervised node classification on graphs.

Other possible downstream tasks on hypergraphs include:

* Hyperedge classification/regression: predict the labels of the hyperedges instead of nodes, similar to edge classification.
* Hyperedge completion: given an incomplete hyperedge, predict which nodes belong to the hyperedge as well.  This is akin to link prediction on graphs.

In [ ]:
import tqdm
from torchmetrics.functional import accuracy

def train(model, optimizer, X, Y, train_mask):
    model.train()
    Y_hat = model(X)
    loss = F.cross_entropy(Y_hat[train_mask], Y[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


def evaluate(model, X, Y, val_mask, test_mask, num_classes):
    model.eval()
    Y_hat = model(X)
    val_acc = accuracy(
        Y_hat[val_mask], Y[val_mask], task="multiclass", num_classes=num_classes
    )
    test_acc = accuracy(
        Y_hat[test_mask],
        Y[test_mask],
        task="multiclass",
        num_classes=num_classes,
    )
    return val_acc, test_acc


H, X, Y, num_classes, train_mask, val_mask, test_mask = load_data()
model = HGNN(H, X.shape[1], num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

with tqdm.trange(500) as tq:
    for epoch in tq:
        train(model, optimizer, X, Y, train_mask)
        val_acc, test_acc = evaluate(
            model, X, Y, val_mask, test_mask, num_classes
        )
        tq.set_postfix(
            {
                "Val acc": f"{val_acc:.5f}",
                "Test acc": f"{test_acc:.5f}",
            },
            refresh=False,
        )

print(f"Test acc: {test_acc:.3f}")

## Extra content: HGNN+

HGNN+ is able to produce both node representations and hyperedge representations.  The idea is to update hyperedge representations with member node representations, and update node representations with incident hyperedge representations.  More specifically, it does so in an alternating manner: the hyperedge representations are updated first, followed by node representation updates.

The HGNN+ paper provided a simple implementation of the framework above as follows:

* The representation of hyperedge $j$ is computed via averaging the member node representations: $z_j \gets \frac{1}{|\{i': i' \in j\}|} \sum_{i': i' \in j} x_{i'}$.
* The representation of a node $i$ is then computed via averaging the incident hyperedge representations followed by a non-linear projection: $x_i \gets \sigma \left(W \cdot \frac{1}{|\{j': i \in j'\}|} \sum_{j': i \in j'} z_{j'}\right)$, where $W$ is a trainable parameter matrix.

Could you implement the HGNN+ below, excluding the non-linear activation function $\sigma$?

*Hint:* think of how to rewrite the update rules above in matrix form.

Say that the incident matrix is $H \in \mathbb{R}^{N \times E}$ where $N$ is the number of nodes and $E$ is the number of hyperedges.
* Hyperedge update: $Z \gets \dots$
* Node update: $X \gets \dots$

In [ ]:
class HypergraphConvPlus(nn.Module):
    def __init__(self, H, in_size, out_size):
        super().__init__()
        # INSERT YOUR CODE HERE...
        
    def forward(self, X):
        # REPLACE WITH YOUR CODE HERE...
        return X